In [36]:
# Have created the temporary account so be carefule of geniun account , 

In [108]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time
import yaml
import pandas as pd
# import boto3
from datetime import datetime, timedelta
import pyperclip  # To access the clipboard

In [109]:
# Initialize WebDriver with or wihtout Headless Mode
def init_driver(chromedriver_path):
    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--disable-web-security")  # Disable web security 
    chrome_options.add_argument("--window-size=1280,1024")
    chrome_options.add_argument("--ignore-certificate-errors")  # Ignore SSL errors  
    chrome_options.add_argument("--allow-insecure-localhost")  # Allow insecure localhost connections
    driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
    wait = WebDriverWait(driver, 10)
    print("driver intiated successfuly")
    return driver, wait

In [ ]:
chromedriver_path = r"D:\coding\freelancing\stockMarket\chromedriver.exe"
driver, wait = init_driver(chromedriver_path)

In [111]:
def login_to_linkedin(username, password):
    """
    Function to log in to LinkedIn.
    """
    driver.get("https://www.linkedin.com")

    try:
        sign_in_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-test-id='home-hero-sign-in-cta']")))
        sign_in_button.click()
        wait.until(EC.presence_of_element_located((By.XPATH, "//input[@id='username']")))

        # Enter your email and password
        email_input = driver.find_element(By.ID, 'username')
        password_input = driver.find_element(By.ID, 'password')

        email_input.send_keys(username)
        password_input.send_keys(password)
        # Locate the checkbox
        remember_me_checkbox = driver.find_element(By.ID, 'rememberMeOptIn-checkbox')

        # Uncheck the "Keep me logged in" checkbox using JavaScript
        if remember_me_checkbox.is_selected():  # Check if the checkbox is currently selected
            driver.execute_script("arguments[0].click();", remember_me_checkbox)  # Click to uncheck it


        # Click the updated "Sign in" button
        login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn__primary--large from__button--floating' and @type='submit']")))
        login_button.click()

        # Wait for the homepage to load after logging in
        wait.until(EC.presence_of_element_located((By.XPATH, "//nav")))

        print("Login successful!")
    except Exception as e:
        print(f"Login error: {e}")

In [ ]:
username = "mary221@oegmail.com"
password = "maria33221"
login_to_linkedin(username, password)

In [32]:
# *************optimised*********

In [172]:
# Function to clean and access the search box  
def clean_search_box():  
    try:  
        search_box = driver.find_element(By.XPATH, "//input[contains(@placeholder, 'Search')]")  
        search_box.clear()  # Clear any existing text  
        return search_box  
    except Exception as e:  
        print(f"Error accessing search box: {e}")  
        return None 
# Function to convert relative time to datetime  
def convert_relative_time(relative_time):  
    match = re.match(r'(\d+)\s+(\w+)\s+ago', relative_time)
    if match:  
        amount = int(match.group(1))  
        unit = match.group(2)  

        if unit.startswith('day'):  
            return datetime.now() - timedelta(days=amount)  
        elif unit.startswith('hour'):  
            return datetime.now() - timedelta(hours=amount)  
        elif unit.startswith('minute'):  
            return datetime.now() - timedelta(minutes=amount)  
        elif unit.startswith('second'):  
            return datetime.now() - timedelta(seconds=amount)  

    return None 

In [ ]:
# dropdown_menu = driver.find_element(By.XPATH, "//div[contains(@class, 'feed-shared-control-menu__content')]")
# print(dropdown_menu.get_attribute('innerHTML'))
# # print(dropdown_menu.get_attribute('outerHTML'))


In [173]:
# Define the tags to search for  
tags = ["#apple", "#tesla"]  

# Set a global link limit for each tag  
MAX_LINKS_PER_TAG = 2  # You can adjust this number to change the limit globally  

# Initialize a list to store post links with associated tags  
post_links = []
# Loop through each tag  
for tag in tags:  
    print(f"🔍 Scraping for tag: {tag}")  
    link_count = 0  
    try: 
        search_box = clean_search_box()  
        if not search_box:  
            print(f"Failed to access search box for tag")  
            continue  

        search_box.send_keys(tag)  
        search_box.send_keys(Keys.RETURN)  

        # Wait for search results to load  
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'search-results')]")))  

        # Filter posts by clicking the "Posts" button  
        try:  
            javascript_click_posts = """  
            var buttons = document.querySelectorAll('button[aria-pressed="false"]');  
            for (var i = 0; i < buttons.length; i++) {  
                if (buttons[i].innerText.trim() === 'Posts') {  
                    buttons[i].click();  
                    break;  
                }  
            }  
            """  
            driver.execute_script(javascript_click_posts)  
            time.sleep(5) 
            print("✅ Applied 'Posts' filter.")
        except Exception as e:  
            print(f"Error applying 'Posts' filter for tag {tag}")  

        # Apply filters: sort by "Latest"  
        # try:  
        #     javascript_click_sort_by = """  
        #     var sortByButton = document.getElementById('searchFilter_sortBy');  
        #     if (sortByButton) {  
        #         sortByButton.click();  
        #     }  
        #     """  
        #     driver.execute_script(javascript_click_sort_by)  

        #     javascript_click_latest = """  
        #     var latestRadioButton = document.getElementById('sortBy-date_posted');  
        #     if (latestRadioButton) {  
        #         latestRadioButton.click();  
        #     }  
        #     """  
        #     driver.execute_script(javascript_click_latest)  

        #     javascript_click_show_results = """  
        #     var showResultsButton = document.querySelector('button[aria-label="Apply current filter to show results"]');  
        #     if (showResultsButton) {  
        #         showResultsButton.click();  
        #     }  
        #     """  
        #     driver.execute_script(javascript_click_show_results)  
        # except Exception as e:  
        #     print(f"Error applying 'Sort by Latest' filter for tag {tag}: {e}")  

        # Collect links from the posts after applying filters  
        posts = driver.find_elements(By.XPATH, "//li[contains(@class, 'artdeco-card')]")  
        for post in posts:  
            try: 
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", post)  
                time.sleep(2) 
  
                menu_button = post.find_element(By.XPATH, ".//button[contains(@class, 'feed-shared-control-menu__trigger')]")  
                 
                WebDriverWait(driver, 3).until(  
                    EC.element_to_be_clickable(menu_button)  
                )    
                menu_button.click()  
                WebDriverWait(driver, 3).until(  
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'artdeco-dropdown__content-inner')]"))  
                )   
                copy_link_option = driver.find_element(By.XPATH, "//h5[text()='Copy link to post']")   
                copy_link_option.click()  
                post_link = pyperclip.paste()
                post_links.append({'tag': tag, 'link': post_link})  
                link_count += 1
                if link_count >= MAX_LINKS_PER_TAG:  
                    print(f"✅ Collected {link_count} links for tag '{tag}'")  
                    break  

            except Exception as e:  
                print(f"Error processing post for tag {tag}")  

        if link_count < MAX_LINKS_PER_TAG:  
            print(f"Only collected {link_count} links for tag: {tag}")  

    except Exception as e:  
        print(f"Error searching for tag {tag}")  
print(f"\n📊 Total number of collected post links: {len(post_links)}") 

🔍 Scraping for tag: #apple
✅ Applied 'Posts' filter.
✅ Collected 2 links for tag '#apple'
🔍 Scraping for tag: #tesla
✅ Applied 'Posts' filter.
✅ Collected 2 links for tag '#tesla'

📊 Total number of collected post links: 4


In [ ]:
# for entry in post_links:  
#     print(f"Tag: {entry['tag']}, Link: {entry['link']}")

In [174]:
for index, entry in enumerate(post_links):  
    post_link = entry['link']  
    tag = entry['tag']  
    print(f"🔍 Scraping post {index + 1} for tag: {tag}")
    driver.get(post_link)  
    time.sleep(2)  # Wait for the page to load  

    try:  
        username_element = driver.find_element(By.XPATH, "(//span[contains(@class, 'update-components-actor__title')]//span[@aria-hidden='true'])[1]")  
        post_username = username_element.text.strip()  

    except Exception as e:  
        print(f"⚠️ Error locating username for post {index + 1}")  

    try:  
        # Extract content  
        content = driver.find_element(By.XPATH, ".//div[contains(@class, 'update-components-text')]//span[contains(@class, 'break-words')]")  
        content = content.text  
        # print("Content:", content)  
    except Exception as e:  
        print(f"⚠️ Error extracting content for post {index + 1}")  
        content = "Content not available"  

    try:  
        date_posted_element = driver.find_element(By.XPATH, "//span[contains(@class, 'update-components-actor__sub-description')]//span[contains(@class, 'visually-hidden')]")  
        relative_time = date_posted_element.text  
        posted_datetime = convert_relative_time(relative_time)  

        # Format date and time  
        if posted_datetime:  
            date_posted = posted_datetime.strftime('%Y-%m-%d')  
            time_posted = posted_datetime.strftime('%H:%M:%S') 
        else:  
            print("⚠️ Could not parse the date posted.")  
            date_posted = None  
            time_posted = None  
    except Exception as e:  
        print(f"Error locating date for post {index + 1}:")  
        date_posted = None  
        time_posted = None  

    comments = driver.find_elements(By.CSS_SELECTOR, ".comments-comment-entity")   
    all_comments = []  
    # Loop through each comment and extract the user name and comment text  
    for comment in comments:  
        try:  
            user_name_element = comment.find_element(By.CSS_SELECTOR, ".comments-comment-meta__description-title")  
            comment_user = user_name_element.text.strip()  
 
            comment_text_element = comment.find_element(By.CSS_SELECTOR, ".comments-comment-item__main-content")  
            user_comment = comment_text_element.text.strip()    
            comment_dict = {  
                "comment_user": comment_user,  
                "user_comment": user_comment  
            }  

            all_comments.append(comment_dict)  

        except Exception as e:  
            print(f"⚠️ Error extracting comment for post {index + 1}") 

    # Store the extracted data in the required format  
    post_data = {  
        'Post Number': index + 1,  
        'Label': "linkedin_data",  
        'Tag': tag,  
        'Username': post_username,  
        'Time Posted': time_posted,  
        'Date': date_posted,  
        'Content': content,  
        'Comments': all_comments  
    }  

    all_post_data.append(post_data)
    print(f"✅ Post {index + 1} scraped successfully!\n")

# Save all post data to a YAML file with the date in the filename  
yaml_filename = f"linkedin_data_{datetime.now().strftime('%Y-%m-%d')}.yaml"  
with open(yaml_filename, 'w', encoding='utf-8') as yaml_file:  
    yaml.dump(all_post_data, yaml_file, allow_unicode=True)  

print(f"✅ Data saved to YAML: {yaml_filename}")

🔍 Scraping post 1 for tag: #apple
⚠️ Could not parse the date posted.
✅ Post 1 scraped successfully!

🔍 Scraping post 2 for tag: #apple
✅ Post 2 scraped successfully!

🔍 Scraping post 3 for tag: #tesla
✅ Post 3 scraped successfully!

🔍 Scraping post 4 for tag: #tesla
✅ Post 4 scraped successfully!

Data has been written to linkedin_data.yaml.
✅ Data saved to YAML: linkedin_data_2025-02-02.yaml


In [176]:

# Close the driver  
driver.quit()  